In [26]:
import pandas as pd
import numpy as np

In [27]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
alldata=train.append(test)

In [28]:
import os
for f in os.listdir('../input'):
    if 'sample' in f or 'train' in f or 'test' in f:
        continue
    tmp=pd.read_csv('../input/'+f)
    alldata=pd.merge(alldata,tmp,on='id',how='left')
    print f,

severity_type.csv log_feature.csv resource_type.csv event_type.csv


In [29]:
alldata=alldata.drop_duplicates(subset='id')
alldata.head()

,fault_severity,id,location,severity_type,log_feature,volume,resource_type,event_type
0,1,14121,location 118,severity_type 2,feature 312,19,resource_type 2,event_type 34
4,0,9320,location 91,severity_type 2,feature 315,200,resource_type 2,event_type 34
8,1,14394,location 152,severity_type 2,feature 221,1,resource_type 2,event_type 35
12,1,8218,location 931,severity_type 1,feature 80,9,resource_type 8,event_type 15
18,0,14804,location 120,severity_type 1,feature 134,1,resource_type 2,event_type 34


In [5]:
for i in alldata.columns.values:
    print i,alldata[i].value_counts().shape

fault_severity (3,)
id (18552,)
location (1126,)
severity_type (5,)
log_feature (272,)
volume (215,)
resource_type (10,)
event_type (49,)


In [10]:
def getfea(ds):
    return ds.map(str).apply(lambda x:x.split()[-1]).astype(float)
for i in alldata.columns.values:
    alldata[i]=getfea(alldata[i])
alldata.head()

,fault_severity,id,location,severity_type,log_feature,volume,resource_type,event_type
0,1,14121,118,2,312,19,2,34
4,0,9320,91,2,315,200,2,34
8,1,14394,152,2,221,1,2,35
12,1,8218,931,1,80,9,8,15
18,0,14804,120,1,134,1,2,34


In [11]:
train=pd.merge(train[['id']],alldata,on='id',how='left')
test=pd.merge(test[['id']],alldata,on='id',how='left')

In [12]:
train.to_csv('train1.csv',index=False)
test.to_csv('test1.csv',index=False)

In [30]:
#alldata['log-volume']=alldata['log_feature'].map(str)+'_'+alldata['volume'].map(str)
alldata['sr']=alldata.severity_type.map(str)+'_'+alldata.resource_type.map(str)
alldata['er']=alldata.event_type.map(str)+'_'+alldata.resource_type.map(str)
alldata['es']=alldata.event_type.map(str)+'_'+alldata.severity_type.map(str)
alldata['lr']=alldata.log_feature.map(str)+'_'+alldata.resource_type.map(str)
alldata['le']=alldata.log_feature.map(str)+'_'+alldata.event_type.map(str)
alldata['ls']=alldata.log_feature.map(str)+'_'+alldata.severity_type.map(str)

In [31]:
for i in alldata.columns.values:
    print i,alldata[i].value_counts().shape

fault_severity (3,)
id (18552,)
location (1126,)
severity_type (5,)
log_feature (272,)
volume (215,)
resource_type (10,)
event_type (49,)
sr (28,)
er (132,)
es (98,)
lr (490,)
le (737,)
ls (451,)


In [32]:
from sklearn.feature_extraction import DictVectorizer
def onehot(dtrain,dtest):
    vec = DictVectorizer()
    X_sparse = vec.fit_transform(dtrain.T.to_dict().values())
    Xt_sparse = vec.transform(dtest.T.to_dict().values())
    return X_sparse,Xt_sparse

In [17]:
train=pd.merge(train[['id']],alldata,on='id',how='left')
test=pd.merge(test[['id']],alldata,on='id',how='left')
X,Xt=onehot(train[['sr']],test[['sr']])
import pickle
print X.shape,Xt.shape
pickle.dump(X,open('X1.p','w'))
pickle.dump(Xt,open('Xt1.p','w'))

(7381, 27) (11171, 27)


In [25]:
train=pd.merge(train[['id']],alldata,on='id',how='left')
test=pd.merge(test[['id']],alldata,on='id',how='left')
X,Xt=onehot(train[['sr','es','er']],test[['sr','es','er']])
import pickle
print X.shape,Xt.shape
pickle.dump(X,open('X2.p','w'))
pickle.dump(Xt,open('Xt2.p','w'))

(7381, 212) (11171, 212)


In [33]:
feas=['sr','es','er','le','lr','ls']
train=pd.merge(train[['id']],alldata,on='id',how='left')
test=pd.merge(test[['id']],alldata,on='id',how='left')
X,Xt=onehot(train[feas],test[feas])
import pickle
print X.shape,Xt.shape
pickle.dump(X,open('X3.p','w'))
pickle.dump(Xt,open('Xt3.p','w'))

(7381, 1407) (11171, 1407)
